# ICO Datasets

In [ ]:
import numpy as np
import pandas as pd
import altair as alt
import re
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce


# ICO Coffee Dataset for 2018
# Units: one-thousand 60-Kg bags
# Prices in USD per Kg

# year = '2018'

def process_data(year):
  '''
  Takes in many datasets and takes the relevant year and transforms the data into
  the desired qualities. The datasets are about the following:
  [consumption, exports, imports, prices paid to workers, and production]

  1) In order to run, simply take the file named Coffee Dataset and import all files within
  2) Then parameterize the function with your desired year

  Input: year --> (str)

  Output: df  --> (pd.DataFrame)
  '''

  # Domestic consumption by all exporting countries
  domestic_consumption = pd.read_csv('domestic-consumption.csv')[['domestic_consumption', year]]
  domestic_consumption = domestic_consumption.rename(columns={'domestic_consumption':'Country', year:'domestic_consumption'})
  domestic_consumption['domestic_consumption'] = domestic_consumption['domestic_consumption'].round(2)

  # Exports of all forms of coffee by all exporting countries
  exports_calendar_year = pd.read_csv('exports-calendar-year.csv')[['exports', year]]
  exports_calendar_year = exports_calendar_year.rename(columns={'exports':'Country', year:'exports'})
  exports_calendar_year['exports'] = exports_calendar_year['exports'].round(2)

  # Total exports by all exporting countries
  exports_crop_year = pd.read_csv('exports-crop-year.csv')[['exports_crop_year', year]]
  exports_crop_year = exports_crop_year.rename(columns={'exports_crop_year':'Country', year:'exports_crop_year'})
  exports_crop_year['exports_crop_year'] = exports_crop_year['exports_crop_year'].round(2)

  # Imports by selected importing countries
  imports = pd.read_csv('imports.csv')[['imports', year]]
  imports = imports.rename(columns={'imports': 'Country', year:'imports'})
  imports['imports'] = imports['imports'].round(2)

  # ICO Composite and group indicator prices
  prices_paid_to_growers = pd.read_csv('prices-paid-to-growers.csv')[['prices_paid_to_growers', year]]
  prices_paid_to_growers = prices_paid_to_growers.rename(columns={'prices_paid_to_growers':'Country', year:'prices_paid_to_growers'})
  prices_paid_to_growers['prices_paid_to_growers'] = prices_paid_to_growers['prices_paid_to_growers'].round(2)

  # Total production by all exporting countries
  total_production = pd.read_csv('total-production.csv')[['total_production', year]]
  total_production = total_production.rename(columns={'total_production':'Country', year:'total_production'})
  total_production['total_production'] = total_production['total_production'].round(2)

  # Gross opening stocks in all exporting countries
  #     This refers to the total amount of coffee available for exportation at the beginning
  #     of a specified period. It includes all coffee stocks held by exporting countries,
  #     which could be stored in warehouses, processing facilities, or any other storage locations.
  gross_opening_stocks = pd.read_csv('gross-opening-stocks.csv')[['gross_opening_stocks', year]]
  gross_opening_stocks = gross_opening_stocks.rename(columns={'gross_opening_stocks':'Country', year:'gross_opening_stocks'})

  datasets = [domestic_consumption,
              exports_calendar_year,
              exports_crop_year,
              imports,
              prices_paid_to_growers,
              total_production,
              gross_opening_stocks]

  # Adding in only the datasets that are of shape (56, 2)
  dfs_to_merge = []
  for df in datasets:
    if df.shape == (56, 2):
      dfs_to_merge.append(df)

  df = reduce(lambda left, right: pd.merge(left, right, on='Country'), dfs_to_merge)
  df.head()

  # Adding the country IDs
  codes = pd.read_csv('world-110m-country-names.txt', delimiter='\t')
  codes = codes.rename(columns={'name':'Country'})
  df = df.merge(codes, on='Country')

  # Adding longitude and latitude
  lat_lon = pd.read_csv('countries_lat_lon.txt', delimiter='\t')
  df = df.merge(lat_lon, on='Country')

  # # Adding world population and continent
  world_pop = pd.read_csv('world_population.csv')
  nineties = ['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
             '1998', '1999']
  two_thousands = ['2000', '2001', '2002', '2003', '2004', '2005',
              '2006', '2007', '2008', '2009']
  twenty_tens = ['2010', '2011', '2012', '2013',
              '2014', '2015', '2016', '2017', '2018']
  if year in nineties:
    world_pop['Avg Decade Population'] = (world_pop['1990 Population'] + world_pop['2000 Population']) / 2
  if year in two_thousands:
    world_pop['Avg Decade Population'] = (world_pop['2000 Population'] + world_pop['2010 Population']) / 2
  if year in twenty_tens:
    world_pop['Avg Decade Population'] = (world_pop['2010 Population'] + world_pop['2020 Population']) / 2
  world_pop = world_pop[['Country/Territory', 'Continent', 'Avg Decade Population', 'Density (per km²)']]
  world_pop = world_pop.rename(columns={'Country/Territory':'Country'})
  df = df.merge(world_pop, on='Country')

  df = df.rename(columns = {'domestic_consumption':'Domestic Consumption',
                            'exports_crop_year':'Exports Crop Year',
                            'total_production':'Total Production',
                            'gross_opening_stocks':'Gross Opening Stocks',
                            'Density (per km²)':'Density'})

  return df

In [ ]:
# Puts all the years dataframes into a dictionary that can be easily traversed
# through and accessed for time series analysis
all_years = ['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
             '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
             '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
             '2014', '2015', '2016', '2017', '2018']

all_sets = {}

for year in all_years:
  all_sets[f'ico_df_{year}'] = process_data(year)
  all_sets[f'ico_df_{year}']['Year'] = int(year)

ico_all_years = pd.concat(all_sets.values(), ignore_index=True)
ico_all_years.shape

In [ ]:
ico_all_years_conts = ico_all_years.groupby(['Continent','Year']).agg('sum').reset_index()

continent_flags = {
    'North America': 'https://upload.wikimedia.org/wikipedia/commons/4/43/Location_North_America.svg',
    'South America': 'https://upload.wikimedia.org/wikipedia/commons/0/0f/South_America_%28orthographic_projection%29.svg',
    'Asia': 'https://upload.wikimedia.org/wikipedia/commons/b/bd/Asia_%28orthographic_projection%29_without_New_Guinea.svg',
    'Africa': 'https://upload.wikimedia.org/wikipedia/commons/8/86/Africa_%28orthographic_projection%29.svg',
    'Oceania': 'https://upload.wikimedia.org/wikipedia/commons/8/88/Oceania_%28centered_orthographic_projection%29.svg'
}

In [ ]:
# Select the hour column to slide on
# years= alt.selection_single(
#     fields=['Year'],
#     bind=alt.binding_range(min=1990, max=2018, step=1, name='Year'),
#     init={'Year': 1990}
# )

dropdown = alt.binding_select(
    options=list(ico_all_years_conts['Year'].unique()),
    name='Select Year ')
years = alt.selection_single(
    fields=['Year'],
    bind=dropdown,
    init={'Year': 1990}
)

# Define population
population = ico_all_years_conts['Avg Decade Population']
ico_all_years_conts['flag_url'] = ico_all_years_conts['Continent'].map(continent_flags)


# Normalize Total Production and Domestic Consumption
ico_all_years_conts['normalized_production'] = (ico_all_years_conts['Total Production'] / population) * 1000000
max_normalized_production = ico_all_years_conts['normalized_production'].max()
ico_all_years_conts['normalized_consumption'] = (ico_all_years_conts['Domestic Consumption'] / population) * 1000000
max_normalized_consumption = ico_all_years_conts['normalized_consumption'].max()


production = alt.Chart(ico_all_years_conts).mark_image(
    width=50, 
    height=50
    ).encode(
    x=alt.X('normalized_production:Q',
            title='Domestic Production',
            scale=alt.Scale(domain=(-5, max_normalized_production))),
    y=alt.Y('normalized_consumption:Q',
            title='Domestic Consumption',
            scale=alt.Scale(domain=(-5, max_normalized_consumption))),
    url='flag_url',
    tooltip=['Continent',
             alt.Tooltip('normalized_production', title='Production'),
             alt.Tooltip('normalized_consumption', title='Consumption')]
    ).properties(
        width=800,
        height=250,
        title={
            'text':'Production vs Consumption',
            'subtitle':'Coffee Production and Consumption per 1,000,000 People'}
    ).add_selection(
        years
    ).transform_filter(
        years
    ).interactive()

# Consumption vs Population
world_bar = alt.Chart(ico_all_years_conts).mark_bar(width=20, xOffset=-10).encode(
    y=alt.Y('Total Production:Q',
            title='Production and Consumption',
            scale=alt.Scale(domain=(0, max(ico_all_years_conts['Total Production'])))),
    x=alt.X('Continent:N',
            title='Continent',
            axis=alt.Axis(labelAngle=-45)),
    color=alt.value('blue'),
    tooltip=['Total Production:Q']
  ).properties(
        width=800,
        height=250,
        title={
            'text':f'Total Production and Consumption Per Continent',
            'subtitle':'Total Coffee Production and Consumption Throughout the World'}
    ).add_selection(
        years
    ).transform_filter(
        years
    )

world_bar2 = alt.Chart(ico_all_years_conts).mark_bar(width=20, xOffset=10).encode(
    y=alt.Y('Domestic Consumption:Q'),
    x=alt.X('Continent:N',
            title='Continent',
            axis=alt.Axis(labelAngle=-45)),
    color=alt.value('orange'),
    tooltip=['Domestic Consumption:Q']
    ).properties(
        width=800,
        height=250
    )

production & (world_bar + world_bar2).add_selection(
        years
    ).transform_filter(
        years
    )



In [ ]:
# Create dataframe by continent
africa = ico_all_years[ico_all_years['Continent'] == 'Africa']
asia = ico_all_years[ico_all_years['Continent'] == 'Asia']
northAmerica = ico_all_years[ico_all_years['Continent'] == 'North America']
oceania = ico_all_years[ico_all_years['Continent'] == 'Oceania']
southAmerica = ico_all_years[ico_all_years['Continent'] == 'South America']

In [ ]:
# URL for all flags in ico_all_years dataframe
africa_flags = {
    'Angola': 'https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg',
    'Burundi': 'https://upload.wikimedia.org/wikipedia/commons/5/50/Flag_of_Burundi.svg',
    'Madagascar': 'https://upload.wikimedia.org/wikipedia/commons/b/bc/Flag_of_Madagascar.svg',
    'Malawi': 'https://upload.wikimedia.org/wikipedia/commons/d/d1/Flag_of_Malawi.svg',
    'Rwanda': 'https://upload.wikimedia.org/wikipedia/commons/1/17/Flag_of_Rwanda.svg',
    'Zimbabwe': 'https://upload.wikimedia.org/wikipedia/commons/6/6a/Flag_of_Zimbabwe.svg',
    'Zambia': 'https://upload.wikimedia.org/wikipedia/commons/0/06/Flag_of_Zambia.svg',
    'Benin': 'https://upload.wikimedia.org/wikipedia/commons/0/0a/Flag_of_Benin.svg',
    'Cameroon': 'https://upload.wikimedia.org/wikipedia/commons/4/4f/Flag_of_Cameroon.svg',
    'Central African Republic': 'https://upload.wikimedia.org/wikipedia/commons/6/6f/Flag_of_the_Central_African_Republic.svg',
    'Equatorial Guinea': 'https://upload.wikimedia.org/wikipedia/commons/3/31/Flag_of_Equatorial_Guinea.svg',
    'Ethiopia': 'https://upload.wikimedia.org/wikipedia/commons/7/71/Flag_of_Ethiopia.svg',
    'Gabon': 'https://upload.wikimedia.org/wikipedia/commons/0/04/Flag_of_Gabon.svg',
    'Ghana': 'https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Ghana.svg',
    'Guinea': 'https://upload.wikimedia.org/wikipedia/commons/e/ed/Flag_of_Guinea.svg',
    'Kenya': 'https://upload.wikimedia.org/wikipedia/commons/4/49/Flag_of_Kenya.svg',
    'Liberia': 'https://upload.wikimedia.org/wikipedia/commons/b/b8/Flag_of_Liberia.svg',
    'Nigeria': 'https://upload.wikimedia.org/wikipedia/commons/7/79/Flag_of_Nigeria.svg',
    'Sierra Leone': 'https://upload.wikimedia.org/wikipedia/commons/1/17/Flag_of_Sierra_Leone.svg',
    'Togo': 'https://upload.wikimedia.org/wikipedia/commons/6/68/Flag_of_Togo.svg',
    'Uganda': 'https://upload.wikimedia.org/wikipedia/commons/4/4e/Flag_of_Uganda.svg'
}

asia_flags = {
    'Indonesia': 'https://upload.wikimedia.org/wikipedia/commons/9/9f/Flag_of_Indonesia.svg',
    'Timor-Leste': 'https://upload.wikimedia.org/wikipedia/commons/2/26/Flag_of_East_Timor.svg',
    'Philippines': 'https://upload.wikimedia.org/wikipedia/commons/9/99/Flag_of_the_Philippines.svg',
    'India': 'https://upload.wikimedia.org/wikipedia/en/4/41/Flag_of_India.svg',
    'Nepal': 'https://upload.wikimedia.org/wikipedia/commons/9/9b/Flag_of_Nepal.svg',
    'Sri Lanka': 'https://upload.wikimedia.org/wikipedia/commons/1/11/Flag_of_Sri_Lanka.svg',
    'Thailand': 'https://upload.wikimedia.org/wikipedia/commons/a/a9/Flag_of_Thailand.svg',
    'Yemen': 'https://upload.wikimedia.org/wikipedia/commons/8/89/Flag_of_Yemen.svg'
}

northAmerica_flags = {
    'Cuba': 'https://upload.wikimedia.org/wikipedia/commons/b/bd/Flag_of_Cuba.svg',
    'Dominican Republic': 'https://upload.wikimedia.org/wikipedia/commons/9/9f/Flag_of_the_Dominican_Republic.svg',
    'Haiti': 'https://upload.wikimedia.org/wikipedia/commons/5/56/Flag_of_Haiti.svg',
    'Costa Rica': 'https://upload.wikimedia.org/wikipedia/commons/b/bc/Flag_of_Costa_Rica.svg',
    'El Salvador': 'https://upload.wikimedia.org/wikipedia/commons/3/34/Flag_of_El_Salvador.svg',
    'Guatemala': 'https://upload.wikimedia.org/wikipedia/commons/e/ec/Flag_of_Guatemala.svg',
    'Honduras': 'https://upload.wikimedia.org/wikipedia/commons/8/82/Flag_of_Honduras.svg',
    'Jamaica': 'https://upload.wikimedia.org/wikipedia/commons/0/0a/Flag_of_Jamaica.svg',
    'Mexico': 'https://upload.wikimedia.org/wikipedia/commons/f/fc/Flag_of_Mexico.svg',
    'Nicaragua': 'https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Nicaragua.svg',
    'Panama': 'https://upload.wikimedia.org/wikipedia/commons/a/ab/Flag_of_Panama.svg'
}

oceania_flags = {
    'Papua New Guinea': 'https://upload.wikimedia.org/wikipedia/commons/e/e3/Flag_of_Papua_New_Guinea.svg'
}

southAmerica_flags = {
    'Brazil': 'https://upload.wikimedia.org/wikipedia/commons/0/05/Flag_of_Brazil.svg',
    'Ecuador': 'https://upload.wikimedia.org/wikipedia/commons/e/e8/Flag_of_Ecuador.svg',
    'Paraguay': 'https://upload.wikimedia.org/wikipedia/commons/2/27/Flag_of_Paraguay.svg',
    'Peru': 'https://upload.wikimedia.org/wikipedia/commons/c/cf/Flag_of_Peru.svg',
    'Colombia': 'https://upload.wikimedia.org/wikipedia/commons/2/21/Flag_of_Colombia.svg',
    'Guyana': 'https://upload.wikimedia.org/wikipedia/commons/9/99/Flag_of_Guyana.svg'
}

In [ ]:
def cons_prod_chart(continent, flags, name=''):
  # Select the hour column to slide on
  # years = alt.selection_single(
  #     fields=['Year'],
  #     bind=alt.binding_range(min=1990, max=2018, step=1, name='Year'),
  #     init={'Year': 1990}
  # )

  dropdown = alt.binding_select(
    options=list(continent['Year'].unique()),
    name='Select Year ')
  years = alt.selection_single(
      fields=['Year'],
      bind=dropdown,
      init={'Year': 1990}
  )

  # Define population
  population = continent['Avg Decade Population']

  # adding in flag image
  continent['flag_url'] = continent['Country'].map(flags)

  # Normalize Total Production and Domestic Consumption
  continent['normalized_production'] = (continent['Total Production'] / population) * 1000000
  max_normalized_production = continent['normalized_production'].max()
  continent['normalized_consumption'] = (continent['Domestic Consumption'] / population) * 1000000
  max_normalized_consumption = continent['normalized_consumption'].max()
  max_population = continent['Density'].max()


  # Production vs Population
  production = alt.Chart(continent).mark_image(
      width=30,
      height=30
      ).encode(
      x=alt.X('normalized_production:Q',
              title='Total Production',
              scale=alt.Scale(domain=(-5, max_normalized_production+3))),
      y=alt.Y('normalized_consumption:Q',
              title='Domestic Consumption',
              scale=alt.Scale(domain=(-5, max_normalized_consumption+3))),
      url='flag_url',
      tooltip=['Country',
              'Continent',
              'Total Production',
              'Domestic Consumption']
      ).properties(
          width=800,
          height=300,
          title={
              'text':'Production vs Consumption',
              'subtitle':'Coffee Production and Consumption per 1,000,000 People'}
      ).add_selection(
          years
      ).transform_filter(
          years
      ).interactive()

  # Consumption vs Population
  world_bar = alt.Chart(continent).mark_bar(width=16, xOffset=-8).encode(
      x=alt.X('Country:N',
              title='Country',
              axis=alt.Axis(labelAngle=-45)),
      y=alt.Y('Total Production:Q',
              title='Production and Consumption',
              scale=alt.Scale(domain=(0, max(continent['Total Production'])))),
      color=alt.value('blue'),
      tooltip=['Total Production:Q']
    ).properties(
          width=800,
          height=250,
          title={
              'text':f'Production and Consumption in {name}',
              'subtitle':'Total Coffee Production and Consumption Per Country'}
      ).add_selection(
          years
      ).transform_filter(
          years
      )

  world_bar2 = alt.Chart(continent).mark_bar(width=16, xOffset=8).encode(
      x=alt.X('Country:N',
              title='Country',
              axis=alt.Axis(labelAngle=-45)),
      y=alt.Y('Domestic Consumption:Q'),
      color=alt.value('orange'),
      tooltip=['Domestic Consumption:Q']
      ).properties(
          width=800,
          height=250
      )

  return production & (world_bar + world_bar2).add_selection(
          years
      ).transform_filter(
          years
      )

In [ ]:
# Africa chart
cons_prod_chart(africa, africa_flags)

In [ ]:
# Asia chart
cons_prod_chart(asia, asia_flags)

In [ ]:
# North America chart
cons_prod_chart(northAmerica, northAmerica_flags)

In [ ]:
# Oceania chart
cons_prod_chart(oceania, oceania_flags)

In [ ]:
# South America chart
cons_prod_chart(southAmerica, southAmerica_flags)